In [1]:
import tensorflow as tf
import numpy as np
import matplotlib
from matplotlib import pyplot as plt

import os

In [2]:
num_classes = 10
img_rows , img_cols , img_ch = 28 , 28 ,1 
input_shape = (img_rows , img_cols , img_ch)

In [3]:
(x_train , y_train ) ,(x_test , y_test )  = tf.keras.datasets.mnist.load_data()

In [5]:
x_train = x_train/ 255.0
x_test  = x_test / 255.0

In [6]:
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import (Input, Activation, Dense, Flatten, Conv2D, 
                                     MaxPooling2D, Dropout, BatchNormalization)

epochs      = 200
batch_size  = 32

In [8]:
@tf.function
def con_layer(x , kernal , bias , s ):
    z = tf.nn.conv2d( x , kernal , strides=[1 , s , s, 1] , padding='VALID')
    z = tf.nn.relu(z + bias )
    return z

In [30]:
class SimpleConv_layer(tf.keras.layers.Layer):
    
    def __init__(self, num_kernals = 32 , kernal_size = (3 , 3 ) , stride= 1):
        
        super().__init__()
        self.num_kernals = num_kernals
        self.kernal_size = kernal_size
        self.stride = stride
        
    def build( self, inputs ):
        num_channels = inputs[-1]
        kernals_shape = (num_channels , *self.kernal_size , self.num_kernals)
        
        glorot_init = tf.initializers.GlorotUniform()
        self.kernels = self.add_weight(name= 'kernal',
                                      shape = kernals_shape ,
                                      initializer = glorot_init ,
                                      trainable = True)
        self.bias = self.add_weight(name = 'bias',
                                   shape = (self.num_kernals,),
                                   initializer = 'random_normal',
                                   trainable = True)
        
    def call(self, inputs):
        return con_layer(inputs , self.kernels , self.bias , self.stride)
    
    def get_config(self):
        {
          'kernel_size': self.kernel_size,
            'strides': self.strides,
            'use_bias': self.use_bias
        }
        

In [31]:
from functools import partial
def l2_reg(coef=1e-2):
    """
    Returns a function computing a weighed L2 norm of a given tensor.
    (this is basically a reimplementation of f.keras.regularizers.l2())
    :param coef:    Weight for the norm
    :return:        Loss function
    """
    return lambda x: tf.reduce_sum(x ** 2) * coef

In [32]:
class ConvWithRegularizers(SimpleConv_layer):
    
    def __init__(self, num_kernels = 32 , 
                kernel_size = (3 ,3 ) ,
                stride =  1, 
                kernel_regularizer = l2_reg(),
                bias_regularizer = None ):
        super().__init__(num_kernels , kernel_size , stride)
        self.kernel_regularizer = kernel_regularizer
        self.bias_regularizer = bias_regularizer
        
    
    def build(self, input_shape):
        super().build(input_shape)
        if self.kernel_regularizer is not None:
            self.add_loss(partial(self.kernel_regularizer, self.kernels))
        if self.bias_regularizer is not None:
            self.add_loss(partial(self.bias_regularizer, self.bias))

In [33]:
conv = ConvWithRegularizers(num_kernels=32, kernel_size=(3, 3), stride=1,
                            kernel_regularizer=l2_reg(1.), bias_regularizer=l2_reg(1.))
conv.build(input_shape=tf.TensorShape((None, 28, 28, 1)))

In [34]:
print(conv.losses)

[<tf.Tensor: id=46, shape=(), dtype=float32, numpy=5.410931>, <tf.Tensor: id=53, shape=(), dtype=float32, numpy=0.08222303>]
